#  Unit Ontology Comparison and Evaluation with ABECTO

This is a project to compare and evaluate unit ontologies using  [ABECTO](https://github.com/fusion-jena/abecto).

## Preparation
First, we start the ABECTO background service. **This might take a few seconds.**

In [ ]:
from abecto.abecto import *
abecto = Abecto("abecto/target/abecto.jar", port = 8080) if 'abecto' not in locals() else abecto
abecto.start()

## Load Ontologies

We are going to comprate the Ontologies [OM 2](https://github.com/HajoRijgersberg/OM), [QUDT 2](http://qudt.org/), and [SWEET 3](https://github.com/ESIPFed/sweet). The RDF files are contained in the project repository. We load them into ABECTO.

In [ ]:
project = abecto.project("Unit Ontologies")
om2 = project.ontology("OM 2")
qudt2 = project.ontology("QUDT 2")
sweet3 = project.ontology("SWEET 3")

om2source = om2.source("RdfFileSourceProcessor").load(open("ontology/om2/om-2.0.rdf"))

qudt2datatype = qudt2.source("RdfFileSourceProcessor").load(open("ontology/qudt2/datatype.ttl"))
qudt2science = qudt2.source("RdfFileSourceProcessor").load(open("ontology/qudt2/science.ttl"))
qudt2qudt = qudt2.source("RdfFileSourceProcessor").load(open("ontology/qudt2/qudt.ttl"))
qudt2dimensionvector = qudt2.source("RdfFileSourceProcessor").load(open("ontology/qudt2/dimensionvector.ttl"))
qudt2quantitykind = qudt2.source("RdfFileSourceProcessor").load(open("ontology/qudt2/quantitykind.ttl"))
qudt2unit = qudt2.source("RdfFileSourceProcessor").load(open("ontology/qudt2/unit.ttl"))
qudt2constant = qudt2.source("RdfFileSourceProcessor").load(open("ontology/qudt2/constant.ttl"))
qudt2discipline = qudt2.source("RdfFileSourceProcessor").load(open("ontology/qudt2/discipline.ttl"))

sweet3Units = sweet3.source("RdfFileSourceProcessor").load(open("ontology/sweet3/reprSciUnits.ttl"))

## Prepare Comparison of Unit Conversions

One important aspect of unit ontologies are the conversion specifications. As the ontologies model them in very different ways, we generate an unified representation of the contained conversions using SPARQL CONSTRUCT queries.

In [ ]:
om2conversions = om2source.into("SparqlConstructProcessor", {"query":""})
qudt2conversions = (qudt2datatype + qudt2science + qudt2qudt + qudt2dimensionvector + qudt2quantitykind + qudt2unit + qudt2constant + qudt2discipline).into("SparqlConstructProcessor", {"query":""})
sweet3conversions = sweet3Units.into("SparqlConstructProcessor", {"query":""})

In [ ]:
om2conversions.setParameter("query", """
PREFIX om: <http://www.ontology-of-units-of-measure.org/resource/om-2/>
CONSTRUCT {
    ?conversion <urn:conversion:unit> ?unit ;
                <urn:conversion:oneEquals> ?oneEquals ;
                <urn:conversion:zeroAt> ?zeroAt ;
                <urn:conversion:referenceUnit> ?referenceUnit .
} WHERE {
    {
        # direct conversion
        ?unit om:hasUnit ?referenceUnit ;
              om:hasFactor ?oneEquals .
        BIND( IRI(CONCAT("urn:om2:conversion:", REPLACE(STR(?unit),".*/",""), ":", REPLACE(STR(?referenceUnit),".*/","") )) AS ?conversion)
        FILTER (STR(?unit) < STR(?referenceUnit))
    } UNION {
        # reverse direct conversion
        ?referenceUnit om:hasUnit ?unit ;
                       om:hasFactor ?inverseOneEquals .
        BIND(1/?inverseOneEquals AS ?oneEquals)
        BIND(IRI(CONCAT("urn:om2:conversion:", REPLACE(STR(?unit),".*/",""), ":", REPLACE(STR(?referenceUnit),".*/","") )) AS ?conversion)
        FILTER (STR(?unit) < STR(?referenceUnit))
    } UNION {
        # conversion by prefix
        ?unit om:hasUnit ?referenceUnit ;
              om:hasPrefix/om:hasFactor ?oneEquals .
        BIND(0 AS ?zeroAt)
        BIND(IRI(CONCAT("urn:om2:conversion:", REPLACE(STR(?unit),".*/",""), ":", REPLACE(STR(?referenceUnit),".*/","") )) AS ?conversion)
        FILTER (STR(?unit) < STR(?referenceUnit))
    } UNION {
        # reverse conversion by prefix
        ?referenceUnit om:hasUnit ?unit ;
                       om:hasPrefix/om:hasFactor ?inverseOneEquals .
        BIND(1/?inverseOneEquals AS ?oneEquals)
        BIND(0 AS ?zeroAt )
        BIND(IRI(CONCAT("urn:om2:conversion:", REPLACE(STR(?unit),".*/",""), ":", REPLACE(STR(?referenceUnit),".*/","") )) AS ?conversion)
        FILTER (STR(?unit) < STR(?referenceUnit))
    } UNION {
        # conversion by scale
        [] om:hasUnit ?unit ;
           om:hasScale/om:hasUnit ?referenceUnit ;
           om:hasFactor ?inverseOneEquals ;
           om:hasOff-Set ?inverseZeroAt ;
        BIND(-1*?inverseZeroAt/?oneEquals AS ?zeroAt)
        BIND(1/?inverseOneEquals AS ?oneEquals)
        BIND(IRI(CONCAT("urn:om2:conversion:", REPLACE(STR(?unit),".*/",""), ":", REPLACE(STR(?referenceUnit),".*/","") )) AS ?conversion)
        FILTER (STR(?unit) < STR(?referenceUnit))
    } UNION {
        # reverse conversion by scale
        [] om:hasUnit ?referenceUnit ;
           om:hasScale/om:hasUnit ?unit ;
           om:hasFactor ?oneEquals ;
           om:hasOff-Set ?zeroAt ;
        BIND(IRI(CONCAT("urn:om2:conversion:", REPLACE(STR(?unit),".*/",""), ":", REPLACE(STR(?referenceUnit),".*/","") )) AS ?conversion)
        FILTER (STR(?unit) < STR(?referenceUnit))
    }
}
""")

In [ ]:
qudt2conversions.setParameter("query", """
PREFIX qudt: <http://qudt.org/schema/qudt/>
CONSTRUCT {
    ?conversion <urn:conversion:unit> ?unit ;
                <urn:conversion:oneEquals> ?oneEquals ;
                <urn:conversion:zeroAt> ?zeroAt ;
                <urn:conversion:referenceUnit> ?referenceUnit .
} WHERE {
    ?unit qudt:hasQuantityKind/^qudt:hasQuantityKind ?referenceUnit .
    BIND( IRI(CONCAT("urn:qudt2:conversion:", REPLACE(STR(?unit),".*/",""), ":", REPLACE(STR(?referenceUnit),".*/","") )) AS ?conversion)
    ?unit qudt:conversionMultiplier ?factor1 .
    ?unit qudt:conversionOffset ?offset1 .
    ?referenceUnit qudt:conversionMultiplier ?factor2 .
    ?referenceUnit qudt:conversionOffset ?offset2 .
    BIND(?factor1/?factor2 AS ?oneEquals)
    BIND(?offset1/?factor2-?offset2 AS ?zeroAt)
    FILTER (STR(?unit) < STR(?referenceUnit))
}
""")

In [ ]:
sweet3conversions.setParameter("query", """
PREFIX sorelm: <http://sweetontology.net/relaMath/>
PREFIX sorelsc: <http://sweetontology.net/relaSci/>
CONSTRUCT {
    ?conversion <urn:conversion:unit> ?unit ;
                <urn:conversion:oneEquals> ?oneEquals ;
                <urn:conversion:zeroAt> ?zeroAt ;
                <urn:conversion:referenceUnit> ?referenceUnit .
} WHERE {
    { 
        ?unit sorelsc:hasBaseUnit ?referenceUnit .
        OPTIONAL { ?unit sorelm:hasScalingNumber ?oneEquals }
        OPTIONAL { ?unit sorelm:hasShiftingNumber ?zeroAt }
        FILTER (BOUND(?oneEquals) || BOUND(?zeroAt) )
        BIND( IRI(CONCAT("urn:sweet3:conversion:", REPLACE(STR(?unit),".*/",""), ":", REPLACE(STR(?referenceUnit),".*/","") )) AS ?conversion)
        FILTER (STR(?unit) < STR(?referenceUnit))
    } UNION {
        ?referenceUnit sorelsc:hasBaseUnit ?unit .
        OPTIONAL { ?referenceUnit sorelm:hasScalingNumber ?inverseOneEquals }
        OPTIONAL { ?referenceUnit sorelm:hasShiftingNumber ?inverseZeroAt }
        FILTER (BOUND(?inverseOneEquals) || BOUND(?inverseZeroAt) )
        BIND(COALESCE(1/?inverseOneEquals, 1) AS ?oneEquals)
        BIND(COALESCE(-1*?inverseZeroAt/?inverseOneEquals, -1*?inverseZeroAt ) AS ?zeroAt)
        BIND( IRI(CONCAT("urn:sweet3:conversion:", REPLACE(STR(?unit),".*/",""), ":", REPLACE(STR(?referenceUnit),".*/","") )) AS ?conversion)
        FILTER (STR(?unit) < STR(?referenceUnit))
    }
}
""")

To increase the chance to find wrong values, we derive implicit conversions for OM 2 and SWEET 3. Due to the modeling approach, this is not required for QUDT 2.

In [ ]:
om2conversionsTransitive = om2conversions.into("SparqlConstructProcessor", {"query":""})
sweet3conversionsTransitive = sweet3conversions.into("SparqlConstructProcessor", {"query":""})

In [ ]:
om2conversionsTransitive.setParameter("query", """
PREFIX : <urn:conversion:>
CONSTRUCT {
    ?conversion :unit ?unit ;
                :oneEquals ?oneEquals ;
                :zeroAt ?zeroAt ;
                :referenceUnit ?referenceUnit .
} WHERE {
    {
        [] :unit ?unit ;
           :oneEquals ?oneEquals1 ;
           :zeroAt ?zeroAt1 ;
           :referenceUnit ?betweenUnit .
        [] :unit ?betweenUnit ;
           :oneEquals ?oneEquals2 ;
           :zeroAt ?zeroAt2 ;
           :referenceUnit ?referenceUnit .
        BIND(?oneEquals1*?oneEquals2 AS ?oneEquals)
        BIND(?zeroAt1/?oneEquals2+?zeroAt2 AS ?zeroAt)
    } UNION {
        [] :unit ?unit ;
           :oneEquals ?oneEquals1 ;
           :zeroAt ?zeroAt1 ;
           :referenceUnit ?betweenUnit .
        [] :unit ?referenceUnit ;
           :oneEquals ?oneEquals2 ;
           :zeroAt ?zeroAt2 ;
           :referenceUnit ?betweenUnit .
        BIND(?oneEquals1/?oneEquals2 AS ?oneEquals)
        BIND(?zeroAt1*?oneEquals2-?zeroAt2 AS ?zeroAt)
    } UNION {
        [] :unit ?betweenUnit ;
           :oneEquals ?oneEquals1 ;
           :zeroAt ?zeroAt1 ;
           :referenceUnit ?unit .
        [] :unit ?betweenUnit ;
           :oneEquals ?oneEquals2 ;
           :zeroAt ?zeroAt2 ;
           :referenceUnit ?referenceUnit .
        BIND(?oneEquals2/?oneEquals1 AS ?oneEquals)
        BIND(?zeroAt2-?zeroAt1/?oneEquals2 AS ?zeroAt)
    }
    BIND( IRI(CONCAT("urn:om2:conversion:", REPLACE(STR(?unit),".*/",""), ":", REPLACE(STR(?referenceUnit),".*/","") )) AS ?conversion)
    FILTER (STR(?unit) < STR(?referenceUnit))
}
""")

In [ ]:
sweet3conversionsTransitive.setParameter("query", """
PREFIX : <urn:conversion:>
CONSTRUCT {
    ?conversion :unit ?unit ;
                :oneEquals ?oneEquals ;
                :zeroAt ?zeroAt ;
                :referenceUnit ?referenceUnit .
} WHERE {
    {
        [] :unit ?unit ;
           :oneEquals ?oneEquals1 ;
           :zeroAt ?zeroAt1 ;
           :referenceUnit ?betweenUnit .
        [] :unit ?betweenUnit ;
           :oneEquals ?oneEquals2 ;
           :zeroAt ?zeroAt2 ;
           :referenceUnit ?referenceUnit .
        BIND(?oneEquals1/?oneEquals2 AS ?oneEquals)
        BIND(?zeroAt1/?oneEquals2+?zeroAt2 AS ?zeroAt)
    } UNION {
        [] :unit ?betweenUnit ;
           :oneEquals ?oneEquals1 ;
           :zeroAt ?zeroAt1 ;
           :referenceUnit ?unit .
        [] :unit ?betweenUnit ;
           :oneEquals ?oneEquals2 ;
           :zeroAt ?zeroAt2 ;
           :referenceUnit ?referenceUnit .
        BIND(1/?oneEquals1/?oneEquals2 AS ?oneEquals)
        BIND(-1*?zeroAt1/?oneEquals2+?zeroAt2 AS ?zeroAt)
    } UNION {
        [] :unit ?unit ;
           :oneEquals ?oneEquals1 ;
           :zeroAt ?zeroAt1 ;
           :referenceUnit ?betweenUnit .
        [] :unit ?referenceUnit ;
           :oneEquals ?oneEquals2 ;
           :zeroAt ?zeroAt2 ;
           :referenceUnit ?betweenUnit .
        BIND(?oneEquals1*?oneEquals2 AS ?oneEquals)
        BIND(?zeroAt1*?oneEquals2-?zeroAt2 AS ?zeroAt)
    }
    BIND( IRI(CONCAT("urn:sweet3:conversion:", REPLACE(STR(?unit),".*/",""), ":", REPLACE(STR(?referenceUnit),".*/","") )) AS ?conversion)
    FILTER (STR(?unit) < STR(?referenceUnit))
}
""")

## Define Categories

Now, we define patterns for the the categories we want to compare.

In [ ]:
om2Categories = om2conversionsTransitive.into("ManualCategoryProcessor", {})
qudt2Categories = qudt2conversions.into("ManualCategoryProcessor", {})
sweet3Categories = sweet3conversionsTransitive.into("ManualCategoryProcessor", {})

In [ ]:
om2Categories.setParameter("patterns",{
"unit": """
    {
        ?unit rdf:type/rdfs:subClassOf* <http://www.ontology-of-units-of-measure.org/resource/om-2/Unit> .
        OPTIONAL { ?unit <http://www.w3.org/2000/01/rdf-schema#label>  ?label }
        OPTIONAL { ?unit <http://www.ontology-of-units-of-measure.org/resource/om-2/symbol> ?symbol }
        OPTIONAL { ?unit <http://www.w3.org/2000/01/rdf-schema#comment> ?definition }
    }
""","conversion": """
    {
        ?conversion <urn:conversion:unit> ?unit ;
                    <urn:conversion:referenceUnit> ?referenceUnit .
        OPTIONAL { ?conversion <urn:conversion:zeroAt> ?zeroAt .}
        OPTIONAL { ?conversion <urn:conversion:oneEquals> ?oneEquals .}
    }
"""})

In [ ]:
qudt2Categories.setParameter("patterns",{
"unit": """
    {
        ?unit rdf:type/rdfs:subClassOf* <http://qudt.org/schema/qudt/Unit> .
        OPTIONAL { ?unit <http://qudt.org/schema/qudt/description> ?definition }
        OPTIONAL { ?unit <http://qudt.org/schema/qudt/symbol> ?symbol }
        OPTIONAL { ?unit <http://www.w3.org/2000/01/rdf-schema#label> |
                         <http://www.w3.org/2004/02/skos/core#prefLabel> ?label .}
    }
""","conversion": """
    {
        ?conversion <urn:conversion:unit> ?unit ;
                    <urn:conversion:referenceUnit> ?referenceUnit .
        OPTIONAL { ?conversion <urn:conversion:zeroAt> ?zeroAt .}
        OPTIONAL { ?conversion <urn:conversion:oneEquals> ?oneEquals .}
    }
"""})

In [ ]:
sweet3Categories.setParameter("patterns",{
"unit": """
    {
        ?unit rdf:type/rdfs:subClassOf* <http://sweetontology.net/reprSciUnits/Unit> .
        OPTIONAL { ?unit <http://www.w3.org/2000/01/rdf-schema#label>  ?label }
        OPTIONAL { ?unit <http://sweetontology.net/relaSci/hasSymbol> ?symbol }
    }
""","conversion": """
    {
        ?conversion <urn:conversion:unit> ?unit ;
                    <urn:conversion:referenceUnit> ?referenceUnit .
        OPTIONAL { ?conversion <urn:conversion:zeroAt> ?zeroAt .}
        OPTIONAL { ?conversion <urn:conversion:oneEquals> ?oneEquals .}
    }
"""})

## Mapping

An comparison requires a mapping of the resources in the ontologies. We use the Jaro Winkler Similarity of the labels to map the units. The conversions will then be mapped based on the related units.

In [ ]:
manualMapping = (om2Categories + qudt2Categories + sweet3Categories).into("ManualMappingProcessor")
labelMapping = manualMapping.into("JaroWinklerMappingProcessor", {
    "threshold": 0.9,
    "case_sensitive": False,
    "category": "unit",
    "variables": ["label"]})
mapping = labelMapping.into("RelationalMappingProcessor", {
    "category": "conversion",
    "variables": ["unit","referenceUnit"]})

## Comparison

Now the ontologies can be compared:
* We generate statisticw of the number of resources and properties per category.
* We search for deviating property values of mapped resources.

In [ ]:
counts = mapping.into("CategoryCountProcessor");
valueDeviations = mapping.into("LiteralDeviationProcessor", {"variables": {"unit": ["symbol"], "conversion" : ["oneEquals", "zeroAt"] }})

## Execution

The comparison pipeline is now reade to be executed.

In [ ]:
execution = project.runAndAwait()

## Reporting

After the pipeline execution succeeded, we display the comparison results.

In [ ]:
execution.measurements()

In [ ]:
execution.deviations()

In [ ]:
execution.issues()

## Stop Background Service

Finally, we stop the ABECTO background service.

In [ ]:
abecto.stop()